A notebook for gathering celebrity chef Tweet data via the Twitter streaming API.

#0. Setup

In [1]:
import sys
import json
import pandas as pd
import datetime
from requests_oauthlib import OAuth1
import tweepy
from pymongo import MongoClient
import requests.packages.urllib3
import logging

In [2]:
auth = tweepy.OAuthHandler(config["consumer_key"],
                           config["consumer_secret"])
auth.set_access_token(config["access_token"],
                      config["access_token_secret"])
api=tweepy.API(auth)

In [3]:
consumer_key = 'DMJRylOrHbiPEynF50H7UjEUN'
consumer_secret = 'CHIjVykOmxN41aZoAA6awiRszB9eue1gZbCYdsHrrXPlRlK938'
access_token = '4718966085-gAePUTbz8oAQjh0KufOo9JnAe6Dp90aWvPQzHIe'
access_token_secret = '8Fy7p1nfQaGC5VQgzBzL6FfNusejzHjkR1elhv2JrFpix'

In [4]:
chefs_and_handles = {
    'Valerie Bertinelli': 'Wolfiesmom',
    'Ina Garten': 'inagarten',
    'Ree Drummond': 'thepioneerwoman',
    'Giada De Laurentiis': 'GDeLaurentiis',
    'Trisha Yearwood': 'trishayearwood',
    'Guy Fieri': 'GuyFieri', 'GuyFieri_ebooks'
    'Robert Irvine': 'RobertIrvine',
    'Alton Brown': 'altonbrown', 'AltonBrownNews'
    'Bobby Flay': 'bflay',
    'Duff Goldman': 'Duff_Goldman', 'DuffsCakemix'
    'Sunny Anderson': 'SunnyAnderson',
    'marcela-valladolid': 'chefmarcela', 'CasaMarcela'
    }

Twitter IDs to follow:

In [5]:
#http://tweeterid.com/
'''
@gdelaurentiis => 20661539
@altonbrown => 293850289
@trishayearwood => 7215082
@wolfiesmom => 33627557
@robertirvine => 34396691
@sunnyanderson => 34436704
@bflay => 27091468
@guyfieri => 27083523
@inagarten => 463204825
@thepioneerwoman => 14709355
@duff_goldman => 313916789
@guyfieri_ebooks => 603413532
@altonbrownnews => 36697827
@duffscakemix => 316881789
@chefmarcela => 22089893
@casamarcela => 1612743247
'''

'\n@gdelaurentiis => 20661539\n@altonbrown => 293850289\n@trishayearwood => 7215082\n@wolfiesmom => 33627557\n@robertirvine => 34396691\n@sunnyanderson => 34436704\n@bflay => 27091468\n@guyfieri => 27083523\n@inagarten => 463204825\n@thepioneerwoman => 14709355\n@duff_goldman => 313916789\n@guyfieri_ebooks => 603413532\n@altonbrownnews => 36697827\n@duffscakemix => 316881789\n@chefmarcela => 22089893\n@casamarcela => 1612743247\n'

#1. Implementation (run on ec2 via Nohup)

In [6]:
requests.packages.urllib3.disable_warnings() # Disable requests warning messages

urllib3_logger = logging.getLogger('urllib3')
urllib3_logger.setLevel(logging.CRITICAL)

logger = logging.getLogger('project_kojak')
hdlr = logging.FileHandler('/home/ben/kojak/kojak_status.log')
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
hdlr.setFormatter(formatter)
logger.addHandler(hdlr)
logger.setLevel(logging.INFO)

# Twitter account key data from twitter_config file
consumer_key = 'DMJRylOrHbiPEynF50H7UjEUN'
consumer_secret = 'CHIjVykOmxN41aZoAA6awiRszB9eue1gZbCYdsHrrXPlRlK938'
access_token = '4718966085-gAePUTbz8oAQjh0KufOo9JnAe6Dp90aWvPQzHIe'
access_token_secret = '8Fy7p1nfQaGC5VQgzBzL6FfNusejzHjkR1elhv2JrFpix'

"""
Modify the tweepy StreamListener class to provide additional functionality.
Every time the stream receives data it will connect to MongoDB, iniate a
MongoDB database, write the data to a json file and and insert the json file into
a collection in the the database.
"""
class StreamListener(tweepy.StreamListener):
    """
    tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API. It allows us to retrieve tweets in real time.
    """
    def on_connect(self):
        """Called when the connection is made"""
        #timestr = strftime("%Y-%m-%d %H:%M:%S", localtime())
        #print(timestr)
        #print("You're connected to the Twitter streaming server...Ctrl-C to stop the stream")
        logger.info("Connected to the Twitter streaming service")

    def on_error(self, status_code):
        """This is called when an error occurs"""
        #timestr = strftime("%Y-%m-%d %H:%M:%S", localtime())
        #print(timestr)
        #print('Error: ' + repr(status_code), 'stream disconnected')
        logger.error(repr(status_code), "Stream disconnected")
        return False

    def on_data(self, data):
        """This will be called each time we receive stream data"""
        client = MongoClient('localhost', 27017)

        # Use kojak database
        db = client.kojak

        # Decode JSON
        datajson = json.loads(data)

        # We only want to store tweets in English
        if 'lang' in datajson and datajson['lang'] == 'en':
            # Store tweet info into the oscartweets collection.
            db.cheftweets.insert(datajson)


if __name__ == '__main__':
    try:
        # Authenticating
        auth1 = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth1.set_access_token(access_token, access_token_secret)

        # Start the stream
        my_listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True))
        streamer = tweepy.Stream(auth=auth1, listener=my_listener)

        # Filter the Twitter stream to capture only tweets related to the chefs
        #streamer.filter(track=chefs_and_handles.values())
        streamer.filter(follow=['20661539', '293850289', '7215082', '33627557', '34396691', 
                                '34436704', '27091468', '27083523', '463204825', '14709355', 
                                '313916789', '603413532', '36697827', '316881789',
                                '22089893', '36980342', '1612743247'])

    except KeyboardInterrupt:
        logger.info("Interrupt received...stopping stream")
        sys.exit()